# Wind Statistics

### Introduction:

The data have been modified to contain some missing values, identified by NaN.  
Using pandas should make this exercise
easier, in particular for the bonus question.

You should be able to perform all of these operations without using
a for loop or other looping construct.


1. The data in 'wind.data' has the following format:

In [1]:
"""
Yr Mo Dy   RPT   VAL   ROS   KIL   SHA   BIR   DUB   CLA   MUL   CLO   BEL   MAL
61  1  1 15.04 14.96 13.17  9.29   NaN  9.87 13.67 10.25 10.83 12.58 18.50 15.04
61  1  2 14.71   NaN 10.83  6.50 12.62  7.67 11.50 10.04  9.79  9.67 17.54 13.83
61  1  3 18.50 16.88 12.33 10.13 11.17  6.17 11.25   NaN  8.50  7.67 12.75 12.71
"""

"Yr Mo Dy   RPT   VAL   ROS   KIL   SHA   BIR   DUB   CLA   MUL   CLO   BEL   MAL\n61  1  1 15.04 14.96 13.17  9.29   NaN  9.87 13.67 10.25 10.83 12.58 18.50 15.04\n61  1  2 14.71   NaN 10.83  6.50 12.62  7.67 11.50 10.04  9.79  9.67 17.54 13.83\n61  1  3 18.50 16.88 12.33 10.13 11.17  6.17 11.25   NaN  8.50  7.67 12.75 12.71\n"

   The first three columns are year, month and day.  The
   remaining 12 columns are average windspeeds in knots at 12
   locations in Ireland on that day.   

   More information about the dataset go [here](wind.desc).

### Step 1. Import the necessary libraries

In [2]:
using CSV
using DataFrames
using Dates
using Statistics
using Downloads

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/06_Stats/Wind_Stats/wind.data)

### Step 3. Assign it to a variable called data and replace the first 3 columns by a proper datetime index.

In [30]:
url = "https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/06_Stats/Wind_Stats/wind.data"
file = Downloads.download(url)
io = replace(read(file, String), r"(\d)-" => s"\1 -") |> IOBuffer

df = CSV.read(io, DataFrame; skipto=2, delim=' ', ignorerepeated=true)
first(df, 3)

Row,Yr,Mo,Dy,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,61,1,1,15.04,14.96,13.17,9.29,NaN,9.87,13.67,10.25,10.83,12.58,18.5,15.04
2,61,1,2,14.71,NaN,10.83,6.5,12.62,7.67,11.5,10.04,9.79,9.67,17.54,13.83
3,61,1,3,18.5,16.88,12.33,10.13,11.17,6.17,11.25,NaN,8.5,7.67,12.75,12.71


### Step 4. Year 2061? Do we really have data from this year? Create a function to fix it and apply it.

In [31]:
df[!, :Yr] = "19" .* string.(df[!, :Yr])
df[!, :Yr] = parse.(Int32, df[!, :Yr])

df[!, "date"] = Date.(df[!, :Yr], df[!, "Mo"], df[!, "Dy"])
df = df[:, Not([:Yr, :Mo, :Dy])]

first(df, 3)

Row,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL,date
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Date
1,15.04,14.96,13.17,9.29,NaN,9.87,13.67,10.25,10.83,12.58,18.5,15.04,1961-01-01
2,14.71,NaN,10.83,6.5,12.62,7.67,11.5,10.04,9.79,9.67,17.54,13.83,1961-01-02
3,18.5,16.88,12.33,10.13,11.17,6.17,11.25,NaN,8.5,7.67,12.75,12.71,1961-01-03


### Step 5. Compute how many values are missing for each location over the entire record.  
#### They should be ignored in all calculations below. 

In [33]:
column_names = names(df)
missing_dict = Dict()

for col in column_names
    try
        total_missing = count(isnan, df[!, col])
        missing_dict[col] = total_missing
    catch
        continue
    end
end

missing_dict

Dict{Any, Any} with 12 entries:
  "RPT" => 6
  "CLA" => 2
  "DUB" => 3
  "MUL" => 3
  "KIL" => 5
  "SHA" => 2
  "MAL" => 4
  "VAL" => 3
  "ROS" => 2
  "BIR" => 0
  "CLO" => 1
  "BEL" => 0

### Step 6. Compute how many non-missing values there are in total.

In [34]:
n_samples = size(df, 1)
column_names = names(df)
nonmissing_dict = Dict()

for col in column_names
    try
        total_nonmissing = n_samples - missing_dict[col]  
        nonmissing_dict[col] = total_nonmissing
    catch
        continue
    end
end

nonmissing_dict

Dict{Any, Any} with 12 entries:
  "RPT" => 6568
  "CLA" => 6572
  "DUB" => 6571
  "MUL" => 6571
  "KIL" => 6569
  "SHA" => 6572
  "MAL" => 6570
  "VAL" => 6571
  "ROS" => 6572
  "BIR" => 6574
  "CLO" => 6573
  "BEL" => 6574

### Step 7. Calculate the mean windspeeds of the windspeeds over all the locations and all the times.
#### A single number for the entire dataset.

In [35]:
numeric_cols = [name for name in names(df) if eltype(df[!, name]) <: Number]
total_temp = Vector{Float64}(undef, length(numeric_cols))
for (i, col) in enumerate(numeric_cols)
    total_temp[i] = sum(df[!, col]) / nonmissing_dict[col]
end

total = sum(total_temp)


NaN

### Step 10. Create a DataFrame called loc_stats and calculate the min, max and mean windspeeds and standard deviations of the windspeeds at each location over all the days 

#### A different set of numbers for each location.

### Step 10. Create a DataFrame called day_stats and calculate the min, max and mean windspeed and standard deviations of the windspeeds across all the locations at each day.

#### A different set of numbers for each day.

### Step 11. Find the average windspeed in January for each location.  
#### Treat January 1961 and January 1962 both as January.

### Step 12. Downsample the record to a yearly frequency for each location.

### Step 13. Downsample the record to a monthly frequency for each location.

### Step 14. Downsample the record to a weekly frequency for each location.

### Step 15. Calculate the min, max and mean windspeeds and standard deviations of the windspeeds across all locations for each week (assume that the first week starts on January 2 1961) for the first 52 weeks.